In [75]:
import os

import numpy  as np
import pandas as pd
import pickle as pkl

from datetime import datetime
from dateutil.tz import tzlocal
from glob     import glob

import pynwb
from pynwb import NWBFile, TimeSeries

from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.utils import StrDataset

from dlab.nwbtools import option234_positions


In [76]:

file_name = 'reach7_07_2024-12-04_20-57-49_008'
root_recording_folder = fr"G:\Grant\neuropixels\kilosort_recordings\{file_name}"

In [77]:

recording_folder = root_recording_folder + r"\Record Node 103\experiment1\recording1\continuous"
if not os.path.exists(recording_folder):
    raise FileNotFoundError(f"Recording folder not found: {recording_folder}")
else:
    print("Recording folder found: ", recording_folder)

Recording folder found:  G:\Grant\neuropixels\kilosort_recordings\reach7_07_2024-12-04_20-57-49_008\Record Node 103\experiment1\recording1\continuous


In [78]:
#Subject
mouse_id = file_name
sex      = 'M'
species  = 'C57BL/6'

birth_date      = datetime(2024,3,22)
experiment_date = datetime(2024,12,4)
age             = f'P{int((experiment_date - birth_date).days/7)}W'

experimenter           = 'Grant Hughes'
experiment_description = 'Reach task: ChR2-Retro Pontine Gray Stimualtion'

# recording_folder = f'F://{mouse_id.lower()}'
int_path         = os.path.join(recording_folder,'intermediates')
int_path

'G:\\Grant\\neuropixels\\kilosort_recordings\\reach7_07_2024-12-04_20-57-49_008\\Record Node 103\\experiment1\\recording1\\continuous\\intermediates'

In [79]:
units_df = pd.read_json(glob(os.path.join(int_path,'*df_units.json'))[0])
units_df.head()

,cluster_id,Amplitude,ContamPct,KSLabel,amp,ch,depth,fr,group,n_spikes,sh,probe,times,amplitudes,template,peak_wv,xpos,ypos
0,0,754.5,25.0,mua,65.201218,2,4116,3.080043,None,15632,0,A,"[244.2904666667, 244.4332666667, 244.833, 244....","[12.4164581299, 11.961356163, 10.6130752563, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
1,1,1115.6,4.4,good,87.563889,2,4116,2.342543,good,11889,0,A,"[251.6506, 252.5949666667, 252.7104666667, 252...","[18.557964325, 9.2583341599, 9.6833286285, 13....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
2,2,3690.7,94.2,mua,125.804901,2,4116,0.129649,None,658,0,A,"[266.358182659, 266.5259159814, 267.5848825791...","[27.0129261017, 34.935344696, 36.8380126953, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
3,3,1138.3,390.0,mua,69.682053,2,4116,0.014186,None,72,0,A,"[328.9849333333, 464.8765666667, 534.9493, 584...","[14.9501924515, 13.834561348, 15.620092392, 12...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
4,4,1137.4,69.8,mua,85.957153,6,4076,0.682133,None,3462,0,A,"[252.5823333333, 252.7126666667, 255.8557, 255...","[14.7697048187, 11.293598175, 13.0035667419, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,80


In [80]:
stim_df = pd.read_json(glob(os.path.join(int_path,'*df_stim.json'))[0])
stim_df.head()

,start_time,stop_time,stimulus,optogenetics_LED_state
0,265.057246,265.057246,tone1_timestamps,0
1,279.803539,279.803539,tone1_timestamps,0
2,294.557752,294.557752,tone1_timestamps,0
3,309.560026,309.560026,tone1_timestamps,0
4,324.059259,324.059259,tone1_timestamps,0


# Assemble NWB

In [81]:
nwbfile = NWBFile(session_description    = 'reach task with pontine gray stimulation',
                  identifier             = f'{mouse_id}_{experiment_date.strftime("%Y-%m-%d")}',  
                  session_start_time     = datetime.now(tzlocal()),
                  experimenter           = experimenter,
                  lab                    = 'Jason Christie Lab',
                  institution            = 'University of Colorado',
                  experiment_description = experiment_description,
                #   session_id             = os.path.basename(recording_folder)
                  )
nwbfile

root pynwb.file.NWBFile at 0x1868978137936
Fields:
  experiment_description: Reach task: ChR2-Retro Pontine Gray Stimualtion
  experimenter: ['Grant Hughes']
  file_create_date: [datetime.datetime(2025, 2, 3, 14, 12, 16, 573039, tzinfo=tzlocal())]
  identifier: reach7_07_2024-12-04_20-57-49_008_2024-12-04
  institution: University of Colorado
  lab: Jason Christie Lab
  session_description: reach task with pontine gray stimulation
  session_start_time: 2025-02-03 14:12:16.573039-05:00
  timestamps_reference_time: 2025-02-03 14:12:16.573039-05:00

In [82]:
probes       = units_df.probe.unique()
probes.sort()

locations    = ['SIM;IP','PG','MoP','VaL','SnR']
descriptions = ['Neuropixels1.0','Neuropixels1.0','Neuropixels2.0','Neuropixels1.0','Neuropixels1.0'] 
device       = nwbfile.create_device(name='DenmanLab_EphysRig1')

for i, probe in enumerate(probes):
    electrode_name = 'probe'+str(i)
    description = descriptions[i]
    location = locations[i]

    electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                     description=description,
                                                     location=location,
                                                     device=device)
    for ch in range(option234_positions.shape[0]):
        nwbfile.add_electrode(x=option234_positions[ch,0],y=0.,z=option234_positions[0,1],imp=0.0,location='none',filtering='high pass 300Hz',group=electrode_group)

In [83]:
nwbfile.add_unit_column('cluster_id', 'cluster id')
nwbfile.add_unit_column('depth', 'the depth of this unit from the pia')
nwbfile.add_unit_column('xpos', 'the x position on probe')
nwbfile.add_unit_column('ypos', 'the y position on probe')
# nwbfile.add_unit_column('template', 'Kilosort template')
nwbfile.add_unit_column('label', 'user label')
nwbfile.add_unit_column('KSlabel', 'Kilosort label')
nwbfile.add_unit_column('KSamplitude', 'Kilosort amplitude')
nwbfile.add_unit_column('KScontamination', 'Kilosort ISI contamination')
nwbfile.add_unit_column('probe', 'probe ID')
nwbfile.add_unit_column('channel', 'channel')
nwbfile.add_unit_column('n_spikes', 'number of spikes')
# nwbfile.add_unit_column('amp', 'amp')

In [84]:
# #Bombcell Outputs
# a = pd.read_csv(r"F:\d5\2023-11-18_18-07-25_d5\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-113.ProbeA\cluster_bc_unitType.tsv",sep='\t')
# b = pd.read_csv(r"F:\d5\2023-11-18_18-07-25_d5\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-113.ProbeB-AP\cluster_bc_unitType.tsv",sep='\t')
# c = pd.read_csv(r"F:\d5\2023-11-18_18-07-25_d5\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-113.ProbeC-AP\cluster_bc_unitType.tsv",sep='\t')

# qmLabels  = list()
# qmLabels += list(a.bc_unitType.values)
# qmLabels += list(b.bc_unitType.values)
# qmLabels += list(c.bc_unitType.values)

# units_df['qmLabel'] = qmLabels
# units_good          = units_df[(units_df.qmLabel == 'GOOD')|(units_df.qmLabel == 'NON-SOMA GOOD')]

In [112]:
units_df.head()

,cluster_id,Amplitude,ContamPct,KSLabel,amp,ch,depth,fr,group,n_spikes,sh,probe,times,amplitudes,template,peak_wv,xpos,ypos
0,0,754.5,25.0,mua,65.201218,2,4116,3.080043,None,15632,0,A,"[244.2904666667, 244.4332666667, 244.833, 244....","[12.4164581299, 11.961356163, 10.6130752563, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
1,1,1115.6,4.4,good,87.563889,2,4116,2.342543,good,11889,0,A,"[251.6506, 252.5949666667, 252.7104666667, 252...","[18.557964325, 9.2583341599, 9.6833286285, 13....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
2,2,3690.7,94.2,mua,125.804901,2,4116,0.129649,None,658,0,A,"[266.358182659, 266.5259159814, 267.5848825791...","[27.0129261017, 34.935344696, 36.8380126953, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
3,3,1138.3,390.0,mua,69.682053,2,4116,0.014186,None,72,0,A,"[328.9849333333, 464.8765666667, 534.9493, 584...","[14.9501924515, 13.834561348, 15.620092392, 12...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,40
4,4,1137.4,69.8,mua,85.957153,6,4076,0.682133,None,3462,0,A,"[252.5823333333, 252.7126666667, 255.8557, 255...","[14.7697048187, 11.293598175, 13.0035667419, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",59,80


In [87]:
labels = {'mua':1,'good':2}

for i,unit_row in units_df.iterrows():

    if unit_row.KSLabel in labels.keys(): 
        ksl=labels[unit_row.KSLabel]

    else: 
        ksl=np.NaN

    nwbfile.add_unit(
                     cluster_id      = unit_row.cluster_id, 
                     spike_times     = unit_row.times,
                     depth           = unit_row.depth,
                     xpos            = unit_row.xpos,
                     ypos            = unit_row.ypos,
                    #  template= unit_row.template,
                     label           = ksl ,
                    # id              = i,
                     KSlabel         = ksl,
                     KSamplitude     = unit_row.Amplitude,
                     KScontamination = unit_row.ContamPct,
                     probe           = unit_row.probe,
                     channel         = unit_row.ch,
                     n_spikes        = unit_row.n_spikes,
                    #  amp             = unit_row.amplitudes,
                    )

cluster_id                                                    0
Amplitude                                                 754.5
ContamPct                                                  25.0
KSLabel                                                     mua
amp                                                   65.201218
ch                                                            2
depth                                                      4116
fr                                                     3.080043
group                                                      None
n_spikes                                                  15632
sh                                                            0
probe                                                         A
times         [244.2904666667, 244.4332666667, 244.833, 244....
amplitudes    [12.4164581299, 11.961356163, 10.6130752563, 1...
template      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
peak_wv       [0.0, 0.0, 0.0, 0.0, 0.0, 

### This could be used to add more data to the nwb if needed

In [88]:
# qlabels = {'NOISE':0,'GOOD':1,'MUA':2,'PC':3,'CF':4}
# labels = {'noise':0,'mua':1,'good':2,'none':4}

# for i,unit_row in units_df.iterrows():
#     print(unit_row)
#     if unit_row.qmLabel in qlabels.keys(): 
#         label=qlabels[unit_row.qmLabel]
#     else: 
#         label=np.NaN
        
#     if unit_row.KSLabel in labels.keys(): 
#         ksl=labels[unit_row.KSLabel]
#     else: 
#         ksl=np.NaN

#     nwbfile.add_unit(id              = i,
#                      cluster_id      = unit_row.cluster_id, 
#                      spike_times     = unit_row.times,
#                      depth           = unit_row.depth,
#                      xpos            = unit_row.xpos,
#                      ypos            = unit_row.ypos,
#                     #  template= unit_row.template,
#                      label           = label ,
#                      KSlabel         = ksl,
#                      KSamplitude     = unit_row.Amplitude,
#                      KScontamination = unit_row.ContamPct,
#                      probe           = unit_row.probe,
#                      channel         = unit_row.ch,
#                      n_spikes        = unit_row.n_spikes,
#                     #  amp             = unit_row.amplitudes,
#                     )

In [89]:
for stim in stim_df.stimulus.unique():
    nwbfile.add_epoch(np.array(stim_df[stim_df.stimulus == stim].start_time.values)[0], 
                      np.array(stim_df[stim_df.stimulus == stim].start_time.values)[-1]+3.,
                      [stim])    

In [91]:
nwbfile.add_trial_column('stimulus', 'the stimulus type during this trial')

#flashCSD
# nwbfile.add_trial_column('flash_luminance', 'the luminance of the flash on this trial. 1=bright,0=dark')
#nwbfile.add_trial_column('optical_Stim', 'optical stim was on or off')
# nwbfile.add_trial_column('uv', 'uv value on this trial')

#gratings_orientation
# nwbfile.add_trial_column('contrast', 'index of each trial into a matrix of images')
# nwbfile.add_trial_column('temporal_frequency', 'index of each trial into a matrix of images')
# nwbfile.add_trial_column('spatial_frequency', 'index of each trial into a matrix of images')
# nwbfile.add_trial_column('orientation', 'index of each trial into a matrix of images')


#arbMatrix and scene flicker
# nwbfile.add_trial_column('stimulus_index', 'index of each trial into a matrix or list of images')

In [92]:
for stim in stim_df.stimulus.unique():
    mean_diff = np.mean(np.diff(stim_df[stim_df.stimulus == stim].start_time.values))
    for i,row in stim_df[stim_df.stimulus == stim].iterrows():
        nwbfile.add_trial(start_time         = row.start_time, 
                          stop_time          = row.start_time + mean_diff,
                          stimulus           = stim,
                          # green              = row.green,
                          # uv                 = row.uv,
                          # contrast           = row.contrast,
                          # temporal_frequency = row.temporal_frequency,
                          # spatial_frequency  = row.spatial_frequency,
                          # orientation        = row.orientation,
                          # stimulus_index     = row.stimulus_index,
                        )

## These cells could be usefull for adding more data to the nwb if needed
- not sure their functuionality yet

In [17]:
# noise_data = pkl.load(open(glob(os.path.join(recording_folder,'experiment_logs/*luminance_noise.pkl'))[0],'rb'))
# times      = stim_df[stim_df.stimulus == 'spatioluminance_noise'].start_time.values

# spatioluminancenoise_ts = TimeSeries(name       = "spatioluminance_noise",
#                                      data       = noise_data['G'],
#                                      unit       = "m",
#                                      timestamps = times,
#                                      )

# nwbfile.add_stimulus_template(spatioluminancenoise_ts)

In [18]:
# noise_data = pkl.load(open(glob(os.path.join(recording_folder,'experiment_logs/*opsin_noise.pkl'))[0],'rb'))
# times      = stim_df[stim_df.stimulus == 'spatiochromatic_noise'].start_time.values

# spatiochromaticnoise_ts = TimeSeries(name       = "spatiochromatic_noise_g",
#                                      data       = noise_data['G'],
#                                      unit       = "m",
#                                      timestamps = times,
#                                      )

# nwbfile.add_stimulus_template(spatiochromaticnoise_ts)

# spatiochromaticnoise_ts = TimeSeries(name       = "spatiochromatic_noise_uv",
#                                      data       = noise_data['UV'],
#                                      unit       = "m",
#                                      timestamps = times,
#                                      )

# nwbfile.add_stimulus_template(spatiochromaticnoise_ts)

In [100]:
import datetime as dt
# THIS SHOULD NOT just point to the folder, but needs to include the name you want the nwb file to be called /nwb_file_name
nwb_save_path = fr"G:\Grant\neuropixels\nwb\{file_name}"

if not os.path.exists(nwb_save_path):
    print("No NWB file found with this name, creating it now: ", nwb_save_path)
else:
    print("NWB save file already exists, saving you from overwriting your data by adding a timestamp to the end of the file name")
    nwb_save_path = nwb_save_path +'___' + dt.datetime.now().strftime("_%Y-%m-%d_%H-%M-%S")


with pynwb.NWBHDF5IO(nwb_save_path, 'w') as io:
    io.write(nwbfile)
    print("NWB file saved as: ", nwb_save_path)

NWB save file already exists, saving you from overwriting your data by adding a timestamp to the end of the file name
NWB file saved as:  G:\Grant\neuropixels\nwb\reach7_07_2024-12-04_20-57-49_008____2025-02-03_14-16-34


In [102]:
io = pynwb.NWBHDF5IO(nwb_save_path, mode='r')
nwb_ = io.read()

In [103]:
df_stim = nwb_.trials.to_dataframe()
# df_opto = nwb_.get_time_intervals('optogenetics_states').to_dataframe()
df_units = nwb_.units.to_dataframe()

In [111]:
df_stim.head()

,start_time,stop_time,stimulus
id,,,
0,265.057246,287.065699,tone1_timestamps
1,279.803539,301.811992,tone1_timestamps
2,294.557752,316.566205,tone1_timestamps
3,309.560026,331.568479,tone1_timestamps
4,324.059259,346.067712,tone1_timestamps


In [109]:
df_units.head()

,cluster_id,depth,xpos,ypos,label,KSlabel,KSamplitude,KScontamination,probe,channel,n_spikes,spike_times
id,,,,,,,,,,,,
0,0,4116,59,40,1,1,754.5,25.0,A,2,15632,"[244.2904666667, 244.4332666667, 244.833, 244...."
1,1,4116,59,40,2,2,1115.6,4.4,A,2,11889,"[251.6506, 252.5949666667, 252.7104666667, 252..."
2,2,4116,59,40,1,1,3690.7,94.2,A,2,658,"[266.358182659, 266.5259159814, 267.5848825791..."
3,3,4116,59,40,1,1,1138.3,390.0,A,2,72,"[328.9849333333, 464.8765666667, 534.9493, 584..."
4,4,4076,59,80,1,1,1137.4,69.8,A,6,3462,"[252.5823333333, 252.7126666667, 255.8557, 255..."


In [107]:
dfA = df_units[df_units.probe=='A'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
dfB = df_units[df_units.probe=='B'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
dfC = df_units[df_units.probe=='C'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
dfD = df_units[df_units.probe=='D'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
dfE = df_units[df_units.probe=='E'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation

print(f'Good units in probe A: {len(dfA)} --> SIM & IP')
print(f'Good units in probe B: {len(dfB)} --> PG')
print(f'Good units in probe C: {len(dfC)} --> Mop')
print(f'Good units in probe D: {len(dfD)} --> VaL')
print(f'Good units in probe E: {len(dfE)} --> SnR')

Good units in probe A: 211 --> SIM & IP
Good units in probe B: 4 --> PG
Good units in probe C: 22 --> Mop
Good units in probe D: 385 --> VaL
Good units in probe E: 9 --> SnR


C:\Users\user\AppData\Local\Temp\ipykernel_26272\2336228700.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfA = df_units[df_units.probe=='A'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\2336228700.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfB = df_units[df_units.probe=='B'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\2336228700.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfC = df_units[df_units.probe=='C'][df_units.KSlabel==2] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\2336228700.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfD = df_units[df_units.probe=='D'][df_units.KSlabel==2] # these

In [108]:
dfA_bad = df_units[df_units.probe=='A'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
dfB_bad = df_units[df_units.probe=='B'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
dfC_bad = df_units[df_units.probe=='C'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
dfD_bad = df_units[df_units.probe=='D'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
dfE_bad = df_units[df_units.probe=='E'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation

print(f'MUA units in probe A: {len(dfA_bad)} --> SIM & IP')
print(f'MUA units in probe B: {len(dfB_bad)} --> PG')
print(f'MUA units in probe C: {len(dfC_bad)} --> Mop')
print(f'MUA units in probe D: {len(dfD_bad)} --> VaL')
print(f'MUA units in probe E: {len(dfE_bad)} --> SnR')


MUA units in probe A: 336 --> SIM & IP
MUA units in probe B: 229 --> PG
MUA units in probe C: 97 --> Mop
MUA units in probe D: 391 --> VaL
MUA units in probe E: 244 --> SnR


C:\Users\user\AppData\Local\Temp\ipykernel_26272\3665535943.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfA_bad = df_units[df_units.probe=='A'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\3665535943.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfB_bad = df_units[df_units.probe=='B'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\3665535943.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfC_bad = df_units[df_units.probe=='C'][df_units.KSlabel==1] # these are the "good" labelled units from phy before curation
C:\Users\user\AppData\Local\Temp\ipykernel_26272\3665535943.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfD_bad = df_units[df_units.probe=='D'][df_units.KSl

### browse with NWBwidget
- Uncomment the line below for nwbwidgets first time use 

In [ ]:
# !pip install nwbwidgets

### load the nwb file

In [ ]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
import dlab.psth_and_raster as psth

In [4]:
#nwb_save_path = 'set/path/to/your/nwb/file/if/you/want/to/load/it/again.nwb'
nwb_save_path = r'G:\Grant\neuropixels\nwb\reach7_07_2024-12-04_20-57-49_008____2025-02-03_14-16-34'
nwb_load_path = nwb_save_path

io = NWBHDF5IO(nwb_load_path, mode='r')
nwb_ = io.read()

In [ ]:
df_stim = nwb_.trials.to_dataframe()
# df_opto = nwb_.get_time_intervals('optogenetics_states').to_dataframe()
df_units = nwb_.units.to_dataframe()

In [5]:
nwb2widget(nwb_)